# Corretoras e Imobiliarias

Vamos pegar as tabelas de todos os corretores e imobiliarias do rio de janeiro

## Recuperando tabela de consultores/imobiliaris

### Importando Dependências

In [4]:
import time
import pandas as pd
import pyautogui
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from io import StringIO

### Lista de Bairros

In [8]:
lista_bairros_zona_norte = [
    "Acari", "Anchieta", "Barros Filho", "Bento Ribeiro", "Brás de Pina", "Campinho", "Cavalcanti", "Cascadura", "Coelho Neto", "Colégio",
    "Complexo do Alemão", "Cordovil", "Costa Barros", "Engenheiro Leal", "Engenho da Rainha", "Guadalupe", "Honório Gurgel", "Inhaúma", "Irajá",
    "Jardim América", "Madureira", "Marechal Hermes", "Oswaldo Cruz", "Parada de Lucas", "Parque Anchieta", "Parque Colúmbia", "Pavuna", "Penha",
    "Penha Circular", "Quintinho Bocaiuva", "Ricardo de Albuquerque", "Rocha Miranda", "Tómas Coelho", "Turiaçu", "Vaz Lobo", "Vicente de Carvalho",
    "Vigário Geral", "Vila da Penha", "Vila Kosmos", "Vista Alegre" 
]
lista_bairros_ilha_gov_leopoldina = [
    "Bonsucesso", "Bancários", "Cacuia", "Cidade Universitária", "Cocotá", "Freguesia", "Galeão", "Jardim Carioca", "Jardim Guanabara", "Maré",
    "Moneró", "Olaria", "Pitangueiras", "Portuguesa", "Praia da Bandeira", "Ramos", "Ribeira", "Tauá", "Zumbi" 
]
lista_bairros_tijuca = [
    "Alto da Boa Vista", "Andaraí", "Grajaú", "Maracanã", "Praça da Bandeira", "Tijuca", "Vila Isabel", 
]
lista_bairros_meier = [
    "Abolição", "Agua Santa", "Cachambi", "Del Castilho", "Encantado", "Engenho de Dentro", "Engenho Novo", "Higienópolis", "Jacaré", "Jacarezinho",
    "Lins de Vasconcelos", "Manguinhos", "Maria da Graça", "Méier", "Piedade", "Pilares", "Riachuelo", "Rocha", "Sampaio", "São Francisco Xavier",
    "Todos os Santos",  
]

lista_bairros = lista_bairros_zona_norte + lista_bairros_ilha_gov_leopoldina + lista_bairros_tijuca + lista_bairros_meier
dict_bairros = {}

### Baixar tabela por bairro

In [9]:
def baixar_tabela_por_bairro(bairro):
    url = 'https://www.crecirj.conselho.net.br/form_pesquisa_cadastro_geral_site.php'
    driver = webdriver.Edge()

    # Maximize a janela do navegador
    driver.maximize_window()
    
    #Abrir pagina
    driver.get(url)
    
    # Localiza os campos do formulário e preenche-os
    campo_bairro = driver.find_element("id", "input-27")
    campo_bairro.send_keys(bairro)

    # Envie o formulário
    campo_bairro.send_keys(Keys.RETURN)

    # Aguarde um tempo para a página carregar completamente
    time.sleep(6)

    # Pular para o final da pagina
    for _ in range(25):  
        pyautogui.press('down')  # Simula pressionar a tecla de seta para baixo
        time.sleep(0.2) 

    # Mova o cursor do mouse para o ponto médio do elemento usando PyAutoGUI
    pyautogui.moveTo(171, 979)

    # Simule um clique do mouse usando PyAutoGUI
    pyautogui.click()

    time.sleep(1)

    pyautogui.click()

    time.sleep(6)

    try:
        # Encontra a tabela na página
        table = driver.find_elements('css selector', 'table')
        table_html = table[0].get_attribute('outerHTML')
    except IndexError:
        print(f"Não existe nenhuma corretora/imobiliaria em {bairro}!")
        print(f"Verifique se o nome do bairro esta certo! {bairro}\n")
        driver.quit()
        return pd.DataFrame()
    else:
        df = pd.read_html(StringIO(table_html))[0]  # Este método retorna uma lista de DataFrames, portanto, pegamos o primeiro
        driver.quit()
        print(bairro, len(df),end='; ')
        return df

### Baixando as tabelas de todos os bairros

In [ ]:
for bairro in lista_bairros:
    tabela_bairro = baixar_tabela_por_bairro(bairro)
    dict_bairros.update({bairro:tabela_bairro})

### Baixando apenas um item

In [ ]:
tabela_bairro_tijuca = baixar_tabela_por_bairro("Tijuca")

dict_bairros.update({"Tijuca":tabela_bairro_tijuca})

### Criando uma cópia para preservar o dicionário original

In [ ]:
dict_bairros_edit = dict_bairros.copy()

### Exibindo quantos corretores/imobiliarias tem em cada bairro

In [ ]:
soma_len = 0
for bairro, tb in dict_bairros_edit.items():
    soma_len += len(tb)
    print(bairro, len(tb))

print("Todos:", soma_len)

### Exibindo quantas corretoras/imobiliarias estão ativas e inativas

In [ ]:
ativos_qnt = 0
inativos_qnt = 0

def verifica_aitvos_inativos(dataframe):
    ativos = 0
    inativos = 0
    for index, linha in dataframe.iterrows():
        if " PF " in  linha["Identificação"]:
            if linha["Situação"] == "ATIVO":
                ativos += 1 
            else: 
                inativos += 1
    return (ativos, inativos)


for bairro, tb in dict_bairros_edit.items():
    novo_ativos_qnt, novo_inativos_qnt = verifica_aitvos_inativos(tb)
    ativos_qnt += novo_ativos_qnt
    inativos_qnt += novo_inativos_qnt

print("Ativos", ativos_qnt)
print("Inativos", inativos_qnt)
print("Total", ativos_qnt + inativos_qnt)

### Filtrando Corretoras/Imobiliarias  (Situação = "ATIVO" e Certidão de Regularidade = "REGULAR")

In [ ]:
def bairros_ativos_disponiveis(bairros):
    novo_dict = {}
    for bairro_nome, bairro_tabela in bairros.items():
        print(bairro_nome)
        if not bairro_tabela.empty:
            nova_tabela = bairro_tabela.loc[bairro_tabela["Situação"] == "ATIVO"]
            nova_tabela = nova_tabela.loc[bairro_tabela["Certidão de Regularidade"] == "REGULAR"]
            novo_dict.update({bairro_nome:nova_tabela})
        else:
            print("Vazio",bairro_nome)
    return novo_dict.copy()

print(len(dict_bairros_edit))
dict_imobiliaria_ativa_disponivel = bairros_ativos_disponiveis(dict_bairros_edit)
print(len(dict_imobiliaria_ativa_disponivel))

### Descobrir CNPJ outra etapa

In [ ]:

def descobrir_cnpj(razao_social):
    url = "https://cnpj.linkana.com"
    driver = webdriver.Edge()
    driver.get(url)
    
    time.sleep(1)

    campo_razao_social = driver.find_element("id", "q")
    campo_razao_social.send_keys(razao_social)

    # Envie o formulário
    campo_razao_social.send_keys(Keys.RETURN)
    time.sleep(5)

    
tabela_acari = dict_imobiliaria_ativa_disponivel['Acari']

display(tabela_acari)

for linha in tabela_acari.itertuples():
    
    razao_social = linha.Identificação
    razao_social = razao_social.replace(" PF P", "")
    
    print(razao_social)

## Descobrir CNPJ

### Sites que realizaremos as buscas

In [7]:
URL_LINKANA = "https://cnpj.linkana.com/"
URL_CNPJBIZ = "https://cnpj.biz/empresas"
URL_ECONODATA = "https://www.econodata.com.br/consulta-empresa"

### Recuperando String de Print da Tela

In [34]:
import easyocr
from PIL import Image

def recuperando_string_print(*coordenadas):
    # Inicializa o EasyOCR para o idioma português
    reader = easyocr.Reader(['pt'])

    # Captura uma região da tela onde o texto está localizado (ajuste as coordenadas conforme necessário)
    x1, y1, width, height = coordenadas
    screenshot = pyautogui.screenshot(region=(x1, y1, width, height))

    # Salva a captura de tela em um arquivo temporário
    screenshot_path = "screenshot_temp.png"
    screenshot.save(screenshot_path)

    # Usa EasyOCR para realizar OCR na imagem
    resultados = reader.readtext(screenshot_path)
    
    # Extrai o texto dos resultados
    texto_copiado = ""
    for resultado in resultados:
        texto_copiado += resultado[1] + " "
    print("Texto:", texto_copiado)
    return texto_copiado.strip()


### Buscar no CNPJ.BIZ

In [37]:
def buscar_cnpj_no_cnpjbiz(nome_empresa):
    pyautogui.PAUSE = 1
    pyautogui.press("win")
    pyautogui.write("edge")
    pyautogui.press("enter")

    time.sleep(2)

    pyautogui.write(URL_CNPJBIZ)
    pyautogui.press("enter")

    time.sleep(5)
    pyperclip.copy(nome_empresa)

    pyautogui.click(1062, 409)
    pyautogui.hotkey("ctrl", "v")
    time.sleep(1)
    pyautogui.press("enter")
    time.sleep(1)

    deu_ruim = recuperando_string_print(639, 235, 300, 400)
    time.sleep(4)
    if "Opa! Alguma coisa saiu errado Desculpe;" in deu_ruim:
        print("Deu ruim")
        print(deu_ruim)
        return False
    else:
        for i in range(5):
            pyautogui.press("down")

        pyautogui.moveTo(445,430)
        texto = recuperando_string_print(445, 430, 300, 200)
        print(texto)
        return texto
    time.sleep(6)

    
cnpj_empresa = buscar_cnpj_no_cnpjbiz("2M INTERMEDIAÇÕES LTDA")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Texto: AN 


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Texto: 24.701.445/0001-67 Rio de Jane 
24.701.445/0001-67 Rio de Jane


## Conectar Google Sheets 

### Importando dependências

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from gspread_dataframe import set_with_dataframe
import gspread

### Criar Planilhas no GOOGLE SHEETS

In [10]:
def criar_planilha_gs(nome, service):
    # Create the spreadsheet
    planilha = {"properties": {"title": nome}}
    planilha = (
        service.spreadsheets()
        .create(body=planilha, fields="spreadsheetId")
        .execute()
    )
    spreadsheet_id = planilha.get("spreadsheetId")
    print(f"Spreadsheet ID: {spreadsheet_id}")
    return spreadsheet_id

### Ler Planilhas no GOOGLE SHEETS

In [11]:
def ler_planilha(id_planilha, service, range="Página1!A:Z"):
    planilha = service.spreadsheets().values().get(spreadsheetId=id_planilha, range=range).execute()
    print(planilha['values'])
    return planilha

### Editar planilhas no GOOGLE SHEETS

In [12]:
def editar_planilha(id_planilha, service, novo_conteudo, range="Pagina1!A"):
    planilha = service.spreadsheets().values().update(
        spreadsheetId=id_planilha,
        range=range, valueInputOption="USER_ENTERED",
        body={'values':novo_conteudo}
    ).execute()
    print("Planilha Editada")
    return planilha

### Transforma Dataframe em Lista

In [13]:
def transforma_dataframe_em_lista(dataframe):
    nova_lista = []
    colunas = list(dataframe.columns)[:-1]
    nova_lista.append(colunas)
    
    dataframe_fillna = dataframe.fillna("")
    for index, linha in dataframe_fillna.iterrows():
        itens_lista = []
        for item in linha:
            print(item)
            itens_lista.append(item)
        nova_lista.append(itens_lista)
    return nova_lista

### Validar Token GOOGLE SHEETS

In [14]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

def validar_token_google_sheets():
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

### Transformar Dataframe em planilha

In [15]:

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
id_planilha = "1AeDuHW7f4-7vBVCgQvMBYqx2ZCTxcO7D0qbzMZ2l8Ps"


def transformar_dataframe_planilha(title, dataframe):
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  try:
    service = build("sheets", "v4", credentials=creds)
    conteudo_para_adicionar = transforma_dataframe_em_lista(dataframe)
    editar_planilha(id_planilha, service, conteudo_para_adicionar, range=f"{title}!A:Z")

  except HttpError as err:
    print(err)



In [ ]:
acari = dict_bairros_edit["Acari"]
#print(list(tijuca.columns)[:-1])

transformar_dataframe_planilha("Acari", acari, id_planilha)
